In [2]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt,butter, lfilter
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler

In [3]:
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()

from eegPinelineDesign import CenterAtPeakOfWindow,Threshold_test,spindle_overlapping_test,used_windows_check,cut_segments


In [4]:
import pickle
with open('single subject.p','rb') as handle:
    result = pickle.load(handle)

In [5]:
result.keys()

dict_keys(['nonspindle time', 'spindle', 'non spindle', 'spindle time'])

In [6]:
result['spindle'].keys()

dict_keys(['suj10_d1final'])

In [7]:
fileName=list(result['spindle'].keys())[0]
channelList=['F3','F4','C3','C4','O1','O2']
X = [];Y=[]
for names in channelList:
    for item in result['spindle'][fileName][names]:
        if item.shape[1] == 3000:

            X.append(item[0,:])
            Y.append(1)
    for item in result['non spindle'][fileName][names]:
        if item.shape[1] == 3000:
            X.append(item[0,:])
            Y.append(0)

X=np.array(X);Y=np.array(Y)

In [8]:
X.shape,Y.shape

((1451, 3000), (1451,))

In [9]:
idx=np.arange(X.shape[0])
GG=np.random.choice(tuple(idx),len(idx),replace=False)
def shuffle(x):
    return sorted(x, key=lambda k: random.random())
GG = shuffle(shuffle(shuffle(shuffle(GG))))
XX=[];YY=[]
for idxx in GG:
    XX.append(X[idxx])
    YY.append(Y[idxx])

from sklearn.preprocessing import normalize
normal_X = normalize(XX)
X_train, X_test, y_train, y_test = train_test_split(normal_X, YY, test_size=0.20)

#print(y_train)

clf =LogisticRegression(penalty='l2',C=.1,tol=10e-9,fit_intercept=True,solver='liblinear',
                                             max_iter=10e7,multi_class='ovr',n_jobs=-1)



clf.fit(X_train,y_train)
print('logistic,1 fold, 80/20 train/test slipt')
print(classification_report(clf.predict(X_test),y_test))
print('within training data set',clf.score(X_train,y_train))

logistic,1 fold, 80/20 train/test slipt
             precision    recall  f1-score   support

          0       0.88      0.92      0.90       136
          1       0.93      0.89      0.91       155

avg / total       0.90      0.90      0.90       291

within training data set 0.924137931034


In [10]:
num=2
file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=num)
channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
raw_filter = eegPinelineDesign.load_data(file_to_read,channelList,11, 16)#spindle pass
raw_alpha=eegPinelineDesign.load_data(file_to_read,channelList,8, 12)#alpha pass
raw_spindle=eegPinelineDesign.load_data(file_to_read,channelList,11, 16)#spindle pass
raw_muscle=eegPinelineDesign.load_data(file_to_read,channelList,30, 40)#
channelList = ['F3','F4','C3','C4','O1','O2']
raw_filter.pick_channels(channelList)

Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    2, 7, 4, 4, 2
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 3 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Band-pass filtering from 11 - 16 Hz
Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
[done]
Ready.
Fitting ICA to data using 8 

<RawEDF  |  n_channels x n_times : 6 x 3602000>

In [11]:
time_label={};resolution = 0.1
for names in channelList:
    time_label[names]=[]

In [12]:
centerPoint = 0+5;cnt = 0
while raw_filter.last_samp/1000 - centerPoint > 1.5:
    if (cnt % 500) == 0:
        print(centerPoint)
    for ii,names in enumerate(channelList):
    
        if Threshold_test(centerPoint,raw_alpha,raw_spindle,raw_muscle,ii):
            tempSegment,timeSpan=cut_segments(raw_filter,centerPoint,ii,windowsize = 1.5)
            normalziedSegment = normalize(tempSegment[0,:3000])#key step!!!
            predictedLabel = clf.predict(normalziedSegment)
            time_label[names].append([centerPoint,predictedLabel])
        else:
            time_label[names].append([centerPoint,0])

    centerPoint += resolution;cnt += 1
print('done')

5
55.00000000000051
104.99999999999831
154.99999999999548
204.99999999999264
254.9999999999898
305.00000000000085
355.0000000000122
405.0000000000236
455.00000000003496
505.0000000000463
555.0000000000576
605.000000000069
655.0000000000804
705.0000000000917
755.0000000001031
805.0000000001145
855.0000000001259
905.0000000001372
955.0000000001486
1005.00000000016
1055.000000000136
1105.0000000000905
1155.000000000045
1204.9999999999995
1254.999999999954
1304.9999999999086
1354.9999999998631
1404.9999999998176
1454.9999999997722
1504.9999999997267
1554.9999999996812
1604.9999999996357
1654.9999999995903
1704.9999999995448
1754.9999999994993
1804.9999999994538
1854.9999999994084
1904.999999999363
1954.9999999993174
2004.999999999272
2054.9999999992265
2104.999999999181
2154.9999999991355
2204.99999999909
2254.9999999990446
2304.999999998999
2354.9999999989536
2404.999999998908
2454.9999999988627
2504.999999998817
2554.9999999987717
2604.9999999987263
2654.999999998681
2704.9999999986353
2

In [13]:
with open('single subject testing.p','wb') as handle:
    pickle.dump(time_label,handle)

In [14]:
lengths = []
for ii, names in enumerate(channelList):
    lengths.append(len(time_label[names]))

In [15]:
time_label_matrix = np.zeros([7,np.min(lengths)])
time_label_matrix[0,:]=np.array(time_label['C4'])[:,0]

In [16]:
for ii, names in enumerate(channelList):
    time_label_matrix[ii+1,:] = np.array(time_label[names])[:min(lengths),1]

In [18]:
pd.DataFrame(time_label_matrix.T).sum()

0    64816078.499977
1        1732.000000
2        2089.000000
3        1891.000000
4        2085.000000
5        2281.000000
6        1973.000000
dtype: float64